In [1]:
from cobra import io
from cobra.core import Metabolite
from cobra import flux_analysis
from Functions import *
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "/home/sanu/Documents/Scripts/git/plantcoremetabolism-model/PlantCoreMetabolism_v1_2_3.xml"
model = io.sbml.read_sbml_model(fname)
display.clear_output()

from sweetlovegroup.transform import fixModelCompatibilityIssueCobra015
model = fixModelCompatibilityIssueCobra015(model,fname)


#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
rxn.add_metabolites({met:1})

In [2]:
import pandas as pd
biomass = pd.DataFrame(data={"":["sSUCROSE_b","GLC_c","FRU_c","Starch_b","Cellulose_b","Xylan_b",
                                 "L_PHOSPHATIDATE_p","PHOSPHATIDYL_CHOLINE_r",
                                 "L_1_PHOSPHATIDYL_ETHANOLAMINE_r","DIACYLGLYCEROL_p",
                                 "Galactosyl_galactosyl_diacyl_glycerols_p",
                                 "D_Galactosyl_12_diacyl_glycerols_p","2_Lysophosphatidylcholines_r",
                                 "Lysophosphatidylglycerols_r","Triacylglycerols_p",
                                 "L_1_PHOSPHATIDYL_GLYCEROL_p","L_1_phosphatidyl_inositols_r",
                                 "SULFOQUINOVOSYLDIACYLGLYCEROL_p","Protein_b",
                                 "sMAL_b","sCIT_b","sFUM_b","ARG_c","HIS_c","LYS_c","sASP_b",
                                 "sGLU_b","sSER_b","THR_c","ASN_c","sGLN_b","CYS_c",
                                 "GLY_c","PRO_c","sALA_b","VAL_c","ILE_c","LEU_c",
                                 "MET_c","PHE_c","TYR_c","TRP_c","sGABA_b","PALMITATE_p",
                                 "CPD_9245_p","CPD_17412_p","CPD_17291_p","STEARIC_ACID_p",
                                 "OLEATE_CPD_p","Octadecadienoate_p","LINOLENIC_ACID_p",
                                 "ARACHIDIC_ACID_p","CPD_16709_p","DOCOSANOATE_p"],
                             "leaf":[0.0]*54,"stem":[0.0]*54,"root":[0.0]*54,"seed":[0.0]*54,},dtype="float64")
biomass = biomass.set_index("")

In [3]:
proteinMW = model.metabolites.get_by_id("Protein_b").formula_weight

cobra/core/metabolite.py:104 UserWarning: 17.2020450391 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 25.9492553613 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 5.7497158698 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 4.9536245179 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:104 UserWarning: 0.1420323181 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)


### Mature leaf

Extract sucrose, starch and glucose diel content from Jeannette et al 2000, Figure 3

<img src="references/Screenshot_20190719_151346.png" height=50% width=50%>

In [4]:
df_matureleaf_data = pd.DataFrame(data={"day":[1,2,3,4],
                                        "sucrose dawn (g/m2)":[1.184615384615384,1.1846153846153795,
                                                               1.4923076923076835,1.5846153846153763],
                                        "sucrose dusk (g/m2)":[3.030769230769228,2.2923076923076877,
                                                               3.323076923076915,3.5538461538461448],
                                        "starch dawn (g/m2)":[0.9009900990099027,0.6831683168316811,
                                                              0.504950495049501,0.9009900990098938],
                                        "starch dusk (g/m2)":[1.8118811881188108,1.594059405940591,
                                                              1.3960396039603875,2.326732673267319],
                                        "glucose dawn (g/m2)":[0.3000000000000027,0.380645161290325,
                                                              0.30000000000000293,0.20322580645161725],
                                        "glucose dusk (g/m2)":[0.5548387096774199,0.3709677419354862,
                                                              0.661290322580645,0.3016129032258099]})
df_matureleaf_data.set_index("day",inplace=True,drop=True)
df_matureleaf_data["Δ sucrose(g/m2)"] = df_matureleaf_data["sucrose dusk (g/m2)"]- \
                                                df_matureleaf_data["sucrose dawn (g/m2)"]
df_matureleaf_data["Δ starch(g/m2)"] = df_matureleaf_data["starch dusk (g/m2)"]- \
                                                df_matureleaf_data["starch dawn (g/m2)"]
df_matureleaf_data["Δ glucose(g/m2)"] = df_matureleaf_data["glucose dusk (g/m2)"]- \
                                                df_matureleaf_data["glucose dawn (g/m2)"]
print("Starch:sucrose ratio=")
print(find_average(df_matureleaf_data["Δ starch(g/m2)"])/find_average(df_matureleaf_data["Δ sucrose(g/m2)"]))
df_matureleaf_data

Starch:sucrose ratio=
0.612778817745


,glucose dawn (g/m2),glucose dusk (g/m2),starch dawn (g/m2),starch dusk (g/m2),sucrose dawn (g/m2),sucrose dusk (g/m2),Δ sucrose(g/m2),Δ starch(g/m2),Δ glucose(g/m2)
day,,,,,,,,,
1,0.300000,0.554839,0.900990,1.811881,1.184615,3.030769,1.846154,0.910891,0.254839
2,0.380645,0.370968,0.683168,1.594059,1.184615,2.292308,1.107692,0.910891,-0.009677
3,0.300000,0.661290,0.504950,1.396040,1.492308,3.323077,1.830769,0.891089,0.361290
4,0.203226,0.301613,0.900990,2.326733,1.584615,3.553846,1.969231,1.425743,0.098387



Extract day-time starch:sucrose accumulation rate ratio from Lunn and Hatch 1995, Figure 1

<img src="references/Screenshot_20190714_104104.png" height=25% width=25%>

In [5]:
df_matureleaf_data_2 = pd.DataFrame(data={"Time":["Dawn","Dusk"],
                                          "Hexose (umol hexose/mgChl)":[20.312499999999986-17.265624999999943,
                                                                        118.51562499999999-112.18749999999999],
                                          "Sucrose (umol hexose/mgChl)":[17.265624999999943-12.109375000000004,
                                                                         112.18749999999999-82.89062499999999],
                                          "Starch (umol hexose/mgChl)":[12.109375000000004,82.89062499999999]})

Delta_sucrose = float(df_matureleaf_data_2["Sucrose (umol hexose/mgChl)"][df_matureleaf_data_2["Time"]=="Dusk"]) - \
float(df_matureleaf_data_2["Sucrose (umol hexose/mgChl)"][df_matureleaf_data_2["Time"]=="Dawn"])

Delta_hexose =float(df_matureleaf_data_2["Hexose (umol hexose/mgChl)"][df_matureleaf_data_2["Time"]=="Dusk"])- \
float(df_matureleaf_data_2["Hexose (umol hexose/mgChl)"][df_matureleaf_data_2["Time"]=="Dawn"])

Delta_starch = float(df_matureleaf_data_2["Starch (umol hexose/mgChl)"][df_matureleaf_data_2["Time"]=="Dusk"])-\
float(df_matureleaf_data_2["Starch (umol hexose/mgChl)"][df_matureleaf_data_2["Time"]=="Dawn"])

df_matureleaf_data_2["Starch (dusk-dawn)"]=["-",Delta_starch]
df_matureleaf_data_2["Sucrose (dusk-dawn)"]=["-",Delta_sucrose]
df_matureleaf_data_2["Hexose (dusk-dawn)"]=["-",Delta_hexose]
df_matureleaf_data_2.set_index("Time",drop=True,inplace=True)
df_matureleaf_data_2

,Hexose (umol hexose/mgChl),Starch (umol hexose/mgChl),Sucrose (umol hexose/mgChl),Starch (dusk-dawn),Sucrose (dusk-dawn),Hexose (dusk-dawn)
Time,,,,,,
Dawn,3.046875,12.109375,5.156250,-,-,-
Dusk,6.328125,82.890625,29.296875,70.7812,24.1406,3.28125


### Leaf

Extract lipid  from Gu et al 2017, Figure 3

<img src="references/Screenshot_20190719_162426.png" height=50% width=50%>

In [6]:
df_leaf_content_1 = pd.DataFrame(data={"DGDG (%lipids)":[19.43217665615142],
                                       "MGDG (%lipids)":[45.78864353312303],
                                       "SQDG (%lipids)":[1.4984227129337468],
                                       "PG (%lipids)":[4.479495268138808],
                                       "PC (%lipids)":[5.047318611987379],
                                       "PE (%lipids)":[0.31545741324921295],
                                       "PI (%lipids)":[2.1608832807570963],
                                       "PA (%lipids)":[0.31545741324921295],
                                       "LPG (%lipids)":[0.3627760252365908]})
df_leaf_content_1

,DGDG (%lipids),LPG (%lipids),MGDG (%lipids),PA (%lipids),PC (%lipids),PE (%lipids),PG (%lipids),PI (%lipids),SQDG (%lipids)
0,19.432177,0.362776,45.788644,0.315457,5.047319,0.315457,4.479495,2.160883,1.498423


Extract biomass composition from Seaver 2015 maize leaf model

In [7]:
#import sbml file (had to remove 4-5 metabolite duplicates)
Sam_model = io.sbml.read_sbml_model("Data/Seaver2015_Evidenced-Maize-Leaf-Model_corrected.sbml")
rxn = Sam_model.reactions.get_by_id("biomass0")

In [8]:
df_leaf_content_2 = pd.DataFrame(data={"mets":["glucose","fructose","xylose","galactose","malate","citrate",
                                               "Arg","His","Lys","Asp","Glu","Ser","Thr","Asn","Gln","Cys",
                                               "Gly","Pro","Ala","Val","Ile","Leu","Met","Phe","Tyr","Trp"],
                                       "stoichiometry from Seaver et al 2015 leaf model":
                                       [abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd19001_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd30321_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd01487_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00724_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00130_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00137_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00051_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00119_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00039_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00041_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00023_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00054_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00161_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00132_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00053_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00084_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00033_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00129_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00035_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00156_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00322_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00107_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00060_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00066_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00069_c0"))),
                                        abs(rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00065_c0")))]})
df_leaf_content_2.set_index("mets",drop=True,inplace=True)
df_leaf_content_2

,stoichiometry from Seaver et al 2015 leaf model
mets,
glucose,0.2469
fructose,0.0363
xylose,0.1654
galactose,0.0346
malate,0.0060
citrate,0.0020
Arg,0.0112
His,0.0044
Lys,0.0119


Gather lignin monomer content from Abiven et al 2011, Table 2

In [9]:
df_leaf_content_3 = pd.DataFrame(data={"tissue":["leaf","stem","root"],
                                       "vanillin":[12.6,23.7,28.0],
                                       "acetovanillone":[1.3,1.3,0.6],
                                       "vanillic acid":[1,10,1],
                                       "syringaldehyde":[3.5,35.8,58.2],
                                       "acetosyringone":[5.2,6,6.5],
                                       "syringic acid":[2.5,3.5,2.3],
                                       "p-coumaric acid":[15.1,64.9,93],
                                       "ferulic acid":[13.7,15.9,22.3]})
df_leaf_content_3.set_index("tissue",drop=True,inplace=True)
df_leaf_content_3

,acetosyringone,acetovanillone,ferulic acid,p-coumaric acid,syringaldehyde,syringic acid,vanillic acid,vanillin
tissue,,,,,,,,
leaf,5.2,1.3,13.7,15.1,3.5,2.5,1,12.6
stem,6.0,1.3,15.9,64.9,35.8,3.5,10,23.7
root,6.5,0.6,22.3,93.0,58.2,2.3,1,28.0


Gather cell wall composition from Hatfield and Marita 2017, Table 1

In [10]:
df_leaf_content_4 = pd.DataFrame(data={"corn internode":[9,10,11,12,13,14],
                                       "Neutral sugars":[441.8,546.9,487.3,522.4,521.6,534],
                                       "Lignin g/kg":[73.6,87.3,88.6,91.5,82.7,87.5],
                                       "Uronosyls":[27.4,24.1,21.6,23.7,23.9,20.8],
                                       "Phenolics":[4.8,5.6,5.8,7.2,7.9,8.0],
                                       "Total cell wall":[547.6,663.8,603.2,644.9,636.0,650.3]})
df_leaf_content_4_std = pd.DataFrame(data={"corn internode":[9,10,11,12,13,14],
                                       "Neutral sugars (% cellwall)":df_leaf_content_4["Neutral sugars"]/ \
                                           df_leaf_content_4["Total cell wall"],
                                       "Lignin (% cellwall)":df_leaf_content_4["Lignin g/kg"]/ \
                                           df_leaf_content_4["Total cell wall"],
                                       "Uronosyls (% cellwall)":df_leaf_content_4["Uronosyls"]/ \
                                           df_leaf_content_4["Total cell wall"],
                                       "Phenolics (% cellwall)":df_leaf_content_4["Phenolics"]/ \
                                           df_leaf_content_4["Total cell wall"]})
df_leaf_content_4_std.set_index("corn internode",drop=True,inplace=True)
df_leaf_content_4_std

,Lignin (% cellwall),Neutral sugars (% cellwall),Phenolics (% cellwall),Uronosyls (% cellwall)
corn internode,,,,
9,0.134405,0.806793,0.008766,0.050037
10,0.131516,0.823893,0.008436,0.036306
11,0.146883,0.807858,0.009615,0.035809
12,0.141882,0.810048,0.011165,0.036750
13,0.130031,0.820126,0.012421,0.037579
14,0.134553,0.821159,0.012302,0.031985


Data generated by Schnable lab, email recieved from Lee on 30/09/2019

In [11]:
df_leaf_content_5 = pd.DataFrame(data={"Moisture (%FW)":[64.44,64.87],
                                       "Ash (%FW)":[5.77,3.66],
                                       "Protein (%FW)":[7.98,7.25],
                                       "Lipid (%FW)":[1.32,1.15],
                                       "Carbohydrate (%FW)":[20.49,23.08]})
print df_leaf_content_5

df_leaf_content_5_std = pd.DataFrame(data={"Ash (mg/gDW)":1000*df_leaf_content_5["Ash (%FW)"]/(100 - df_leaf_content_5["Moisture (%FW)"]),
                                           "Protein (mg/gDW)":1000*df_leaf_content_5["Protein (%FW)"]/(100 - df_leaf_content_5["Moisture (%FW)"]),
                                           "Lipid (mg/gDW)":1000*df_leaf_content_5["Lipid (%FW)"]/(100 - df_leaf_content_5["Moisture (%FW)"]),
                                           "Carbohydrate (mg/gDW)":1000*df_leaf_content_5["Carbohydrate (%FW)"]/(100 - df_leaf_content_5["Moisture (%FW)"])})
df_leaf_content_5_std

   Ash (%FW)  Carbohydrate (%FW)  Lipid (%FW)  Moisture (%FW)  Protein (%FW)
0       5.77               20.49         1.32           64.44           7.98
1       3.66               23.08         1.15           64.87           7.25


,Ash (mg/gDW),Carbohydrate (mg/gDW),Lipid (mg/gDW),Protein (mg/gDW)
0,162.260967,576.209224,37.120360,224.409449
1,104.184458,656.988329,32.735554,206.376317


### Summary

- Data generated by Schnable lab was used to gather basic composition
- Lipid composition from Gu et al 2017 was used
- Carbohydrate composition from Seaver et al 2015 was used
- biomass composition avialable in mg/gDW

In [12]:
temp = "stoichiometry from Seaver et al 2015 leaf model"
totalCarb = df_leaf_content_2.at["glucose",temp]+df_leaf_content_2.at["fructose",temp]+\
            df_leaf_content_2.at["xylose",temp]+df_leaf_content_2.at["galactose",temp]

In [13]:
biomass.at["GLC_c","leaf"]=find_average(df_leaf_content_5_std["Carbohydrate (mg/gDW)"])*df_leaf_content_2.at["glucose",temp]/totalCarb
biomass.at["FRU_c","leaf"]=find_average(df_leaf_content_5_std["Carbohydrate (mg/gDW)"])*df_leaf_content_2.at["fructose",temp]/totalCarb
biomass.at["Cellulose_b","leaf"]=find_average(df_leaf_content_5_std["Carbohydrate (mg/gDW)"])*df_leaf_content_2.at["xylose",temp]/totalCarb + \
                                find_average(df_leaf_content_5_std["Carbohydrate (mg/gDW)"])*df_leaf_content_2.at["galactose",temp]/totalCarb
biomass.at["Protein_b","leaf"]=find_average(df_leaf_content_5_std["Protein (mg/gDW)"])
biomass.at["Galactosyl_galactosyl_diacyl_glycerols_p","leaf"]=find_average(df_leaf_content_5_std["Lipid (mg/gDW)"])*df_leaf_content_1["DGDG (%lipids)"]/100
biomass.at["D_Galactosyl_12_diacyl_glycerols_p","leaf"]=find_average(df_leaf_content_5_std["Lipid (mg/gDW)"])*df_leaf_content_1["MGDG (%lipids)"]/100
biomass.at["SULFOQUINOVOSYLDIACYLGLYCEROL_p","leaf"]=find_average(df_leaf_content_5_std["Lipid (mg/gDW)"])*df_leaf_content_1["SQDG (%lipids)"]/100
biomass.at["L_1_PHOSPHATIDYL_GLYCEROL_p","leaf"]=find_average(df_leaf_content_5_std["Lipid (mg/gDW)"])*df_leaf_content_1["PG (%lipids)"]/100
biomass.at["PHOSPHATIDYL_CHOLINE_r","leaf"]=find_average(df_leaf_content_5_std["Lipid (mg/gDW)"])*df_leaf_content_1["PC (%lipids)"]/100
biomass.at["L_1_PHOSPHATIDYL_ETHANOLAMINE_r","leaf"]=find_average(df_leaf_content_5_std["Lipid (mg/gDW)"])*df_leaf_content_1["PE (%lipids)"]/100
biomass.at["L_1_phosphatidyl_inositols_r","leaf"]=find_average(df_leaf_content_5_std["Lipid (mg/gDW)"])*df_leaf_content_1["PI (%lipids)"]/100
biomass.at["L_PHOSPHATIDATE_p","leaf"]=find_average(df_leaf_content_5_std["Lipid (mg/gDW)"])*df_leaf_content_1["PA (%lipids)"]/100
biomass.at["Lysophosphatidylglycerols_r","leaf"]=find_average(df_leaf_content_5_std["Lipid (mg/gDW)"])*df_leaf_content_1["LPG (%lipids)"]/100

### Stem



Gather sucrose content from Sekhon et al 2016, Figure 2

<img src="references/Screenshot_20190723_163940.png" height=50% width=50%>

In [14]:
df_stem_content_1 = pd.DataFrame(data={"Type":range(1,19),
                                       "Sucrose (g/100gDW)":[7.07006369426752,7.26114649681529,10.796178343949,
                                                             8.88535031847134,6.30573248407643,5.44585987261146,
                                                             8.02547770700637,9.7452229299363,6.01910828025478,
                                                             4.39490445859873,9.17197452229299,7.07006369426751,
                                                             6.97452229299363,6.78343949044586,6.78343949044586,
                                                             6.97452229299363,6.78343949044586,9.93630573248408]})
df_stem_content_1.set_index("Type",drop=True,inplace=True)
df_stem_content_1["Sucrose (mg/gDW)"]=df_stem_content_1["Sucrose (g/100gDW)"]*10
df_stem_content_1

,Sucrose (g/100gDW),Sucrose (mg/gDW)
Type,,
1,7.070064,70.700637
2,7.261146,72.611465
3,10.796178,107.961783
4,8.885350,88.853503
5,6.305732,63.057325
6,5.445860,54.458599
7,8.025478,80.254777
8,9.745223,97.452229
9,6.019108,60.191083


In [15]:
print("Average sucrose content (mg/gDW) ="+str(find_average(df_stem_content_1["Sucrose (mg/gDW)"])))

Average sucrose content (mg/gDW) =74.6815286624


Gather glucose content from Sekhon et al 2016, Figure 2

<img src="references/Screenshot_20190725_092838.png" height=50% width=50%>

In [16]:
df_stem_content_2 = pd.DataFrame(data={"Type":range(1,19),
                                       "Glucose (g/100gDW)":[8.91304347826087,9.56521739130435,
                                                             12.8260869565217,9.93788819875777,
                                                             7.51552795031056,6.67701863354037,
                                                             9.6583850931677,11.7080745341615,
                                                             7.14285714285714,6.11801242236025,
                                                             11.4285714285714,9.00621118012422,
                                                             8.81987577639751,8.35403726708075,
                                                             8.4472049689441,9.37888198757764,
                                                             7.79503105590062,11.5217391304348]})
df_stem_content_2.set_index("Type",drop=True,inplace=True)
df_stem_content_2["Glucose (mg/gDW)"]=df_stem_content_2["Glucose (g/100gDW)"]*10
df_stem_content_2

,Glucose (g/100gDW),Glucose (mg/gDW)
Type,,
1,8.913043,89.130435
2,9.565217,95.652174
3,12.826087,128.260870
4,9.937888,99.378882
5,7.515528,75.155280
6,6.677019,66.770186
7,9.658385,96.583851
8,11.708075,117.080745
9,7.142857,71.428571


In [17]:
print("Average glucose content (mg/gDW) ="+str(find_average(df_stem_content_2["Glucose (mg/gDW)"])))

Average glucose content (mg/gDW) =91.5631469979


Gather starch content from Sekhon et al 2016, Figure S3

<img src="references/Screenshot_20190725_085846.png" height=50% width=50%>

In [18]:
df_stem_content_3 = pd.DataFrame(data={"Type":range(1,19),
                                       "Starch (g/100gDW)":[0.402597402597402,0.558441558441559,
                                                            1.06493506493507,0.636363636363637,
                                                            0.558441558441559,0.441558441558441,
                                                            0.324675324675324,0.558441558441559,
                                                            0.207792207792208,0.207792207792208,
                                                            0.636363636363637,0.168831168831169,
                                                            0.207792207792208,0.324675324675326,
                                                            0.402597402597402,0.402597402597402,
                                                            0.441558441558443,0.987012987012987]})
df_stem_content_3.set_index("Type",drop=True,inplace=True)
df_stem_content_3["Starch (mg/gDW)"]=df_stem_content_3["Starch (g/100gDW)"]*10
df_stem_content_3

,Starch (g/100gDW),Starch (mg/gDW)
Type,,
1,0.402597,4.025974
2,0.558442,5.584416
3,1.064935,10.649351
4,0.636364,6.363636
5,0.558442,5.584416
6,0.441558,4.415584
7,0.324675,3.246753
8,0.558442,5.584416
9,0.207792,2.077922


In [19]:
print("Average starch content (mg/gDW) ="+str(find_average(df_stem_content_3["Starch (mg/gDW)"])))

Average starch content (mg/gDW) =4.74025974026


Gather cell wall 6C content from Sekhon et al 2016, Figure 4

<img src="references/Screenshot_20190725_092056.png" height=50% width=50%>

In [20]:
df_stem_content_4 = pd.DataFrame(data={"Type":range(1,19),
                                       "Cell wall glucose (g/100gDW)":[19.1121495327103,16.1214953271028,8.55140186915888,
                                                            7.14953271028037,7.14953271028037,6.96261682242991,
                                                            12.0093457943925,11.2616822429907,11.5420560747664,
                                                            12.6635514018692,11.5420560747664,12.1028037383178,
                                                            12.1028037383178,10.2336448598131,10.7009345794393,
                                                            11.2616822429907,8.83177570093458,5.84112149532711]})
df_stem_content_4.set_index("Type",drop=True,inplace=True)
df_stem_content_4["Cell wall glucose (mg/gDW)"]=df_stem_content_4["Cell wall glucose (g/100gDW)"]*10
df_stem_content_4

,Cell wall glucose (g/100gDW),Cell wall glucose (mg/gDW)
Type,,
1,19.112150,191.121495
2,16.121495,161.214953
3,8.551402,85.514019
4,7.149533,71.495327
5,7.149533,71.495327
6,6.962617,69.626168
7,12.009346,120.093458
8,11.261682,112.616822
9,11.542056,115.420561


In [21]:
print("Average cell wall glucose (mg/gDW) ="+str(find_average(df_stem_content_4["Cell wall glucose (mg/gDW)"])))

Average cell wall glucose (mg/gDW) =108.411214953


Gather cell wall 5C content from Sekhon et al 2016, Figure 5

<img src="references/Screenshot_20190725_092127.png" height=50% width=50%>

In [22]:
df_stem_content_5 = pd.DataFrame(data={"Type":range(1,19),
                                       "Cell wall pentose (g/100gDW)":[7.7037037037037,6.48148148148148,
                                                                       2.62962962962963,2.88888888888889,
                                                                       2.74074074074074,2.7037037037037,
                                                                       3.62962962962963,3.40740740740741,
                                                                       4.03703703703703,4.25925925925926,
                                                                       3.37037037037037,3.74074074074074,
                                                                       3.51851851851852,3.81481481481481,
                                                                       3.2962962962963,4.03703703703704,
                                                                       2.74074074074074,1.92592592592593]})
df_stem_content_5.set_index("Type",drop=True,inplace=True)
df_stem_content_5["Cell wall pentose (mg/gDW)"]=df_stem_content_5["Cell wall pentose (g/100gDW)"]*10
df_stem_content_5

,Cell wall pentose (g/100gDW),Cell wall pentose (mg/gDW)
Type,,
1,7.703704,77.037037
2,6.481481,64.814815
3,2.629630,26.296296
4,2.888889,28.888889
5,2.740741,27.407407
6,2.703704,27.037037
7,3.629630,36.296296
8,3.407407,34.074074
9,4.037037,40.370370


In [23]:
print("Average cell wall pentose (mg/gDW) ="+str(find_average(df_stem_content_5["Cell wall pentose (mg/gDW)"])))

Average cell wall pentose (mg/gDW) =37.1810699588


Data generated by Schnable lab, email recieved from Lee on 30/09/2019

In [24]:
df_stem_content_6 = pd.DataFrame(data={"Moisture (%FW)":[78.46,77.89],
                                       "Ash (%FW)":[0.20,0.28],
                                       "Protein (%FW)":[0.89,.73],
                                       "Lipid (%FW)":[0,0],
                                       "Carbohydrate (%FW)":[20.46,18.62]})
print df_stem_content_6

df_stem_content_6_std = pd.DataFrame(data={"Ash (mg/gDW)":1000*df_stem_content_6["Ash (%FW)"]/(100 - df_stem_content_6["Moisture (%FW)"]),
                                           "Protein (mg/gDW)":1000*df_stem_content_6["Protein (%FW)"]/(100 - df_stem_content_6["Moisture (%FW)"]),
                                           "Lipid (mg/gDW)":1000*df_stem_content_6["Lipid (%FW)"]/(100 - df_stem_content_6["Moisture (%FW)"]),
                                           "Carbohydrate (mg/gDW)":1000*df_stem_content_6["Carbohydrate (%FW)"]/(100 - df_stem_content_6["Moisture (%FW)"])})
df_stem_content_6_std

   Ash (%FW)  Carbohydrate (%FW)  Lipid (%FW)  Moisture (%FW)  Protein (%FW)
0       0.20               20.46            0           78.46           0.89
1       0.28               18.62            0           77.89           0.73


,Ash (mg/gDW),Carbohydrate (mg/gDW),Lipid (mg/gDW),Protein (mg/gDW)
0,9.285051,949.860724,0.0,41.318477
1,12.663953,842.152872,0.0,33.016735


### Summary

- Sugar, glucose, starch, C6 and C5 cellwall sugars data available in Sekhon et al 2016
- unable to use lignin monomer content without knowing lignin content
- Protein content available from data generated by Schnable lab

### Issues

- CHO content according to Schnable data is 900 mg/gDW (approx) while based on Sekhon et al 2016 data, this is 316 mg/gDW


In [25]:
biomass.at["sSUCROSE_b","stem"]=find_average(df_stem_content_1["Sucrose (mg/gDW)"])
biomass.at["GLC_c","stem"]=find_average(df_stem_content_2["Glucose (mg/gDW)"])
biomass.at["Starch_b","stem"]=find_average(df_stem_content_3["Starch (mg/gDW)"])
biomass.at["Cellulose_b","stem"]=find_average(df_stem_content_4["Cell wall glucose (mg/gDW)"])
biomass.at["Xylan_b","stem"]=find_average(df_stem_content_5["Cell wall pentose (mg/gDW)"])
biomass.at["Protein_b","stem"]=find_average(df_stem_content_6_std["Protein (mg/gDW)"])

### Root

Extract amino acid and organic acid content from Carvalhais et al 2011 Figure 1

<img src="references/Screenshot_20190725_131713.png" height=50% width=50%>

In [26]:
df_exudate_content_1 = pd.DataFrame(data={"metabolites":["Asp","Glu","Ser",
                                                       "Gln","Thr","Ala",
                                                       "Tyr","Ile","Lys",
                                                       "GABA","Mal","Cit","tAco"],
                                        "Exudation rate (pmol/gFWroot/hr)":[70.1901140684411,63.8022813688213,
                                                                            39.1634980988593,49.8098859315589,
                                                                            10.5703422053232,28.212927756654,
                                                                            6.31178707224334,12.6996197718631,
                                                                            3.57414448669201,11.1787072243346,
                                                                            231.90184049079647,
                                                                            27.914110429447106,
                                                                            380.67484662576635]})
df_exudate_content_1.set_index("metabolites",drop=True,inplace=True)
df_exudate_content_1

,Exudation rate (pmol/gFWroot/hr)
metabolites,
Asp,70.190114
Glu,63.802281
Ser,39.163498
Gln,49.809886
Thr,10.570342
Ala,28.212928
Tyr,6.311787
Ile,12.699620
Lys,3.574144


Extract sucrose, glucose and fructose content of seminal roots of maize seedlings from Ogawa et al 2009, Figure 1

<img src="references/Screenshot_20190725_155512.png" height=50% width=50%>

Because the root are from seedlings, data is declared irrelevant

Data generated by Schnable lab, email recieved from Lee on 30/09/2019

In [27]:
df_root_content_1 = pd.DataFrame(data={"Moisture (%FW)":[83.85,83.18],
                                       "Ash (%FW)":[2.14,1.55],
                                       "Protein (%FW)":[1.06,0.94],
                                       "Lipid (%FW)":[0,0.09],
                                       "Carbohydrate (%FW)":[12.96,14.24]})
print df_root_content_1

df_root_content_1_std = pd.DataFrame(data={"Ash (mg/gDW)":1000*df_root_content_1["Ash (%FW)"]/(100 - df_root_content_1["Moisture (%FW)"]),
                                           "Protein (mg/gDW)":1000*df_root_content_1["Protein (%FW)"]/(100 - df_root_content_1["Moisture (%FW)"]),
                                           "Lipid (mg/gDW)":1000*df_root_content_1["Lipid (%FW)"]/(100 - df_root_content_1["Moisture (%FW)"]),
                                           "Carbohydrate (mg/gDW)":1000*df_root_content_1["Carbohydrate (%FW)"]/(100 - df_root_content_1["Moisture (%FW)"])})
df_root_content_1_std

   Ash (%FW)  Carbohydrate (%FW)  Lipid (%FW)  Moisture (%FW)  Protein (%FW)
0       2.14               12.96         0.00           83.85           1.06
1       1.55               14.24         0.09           83.18           0.94


,Ash (mg/gDW),Carbohydrate (mg/gDW),Lipid (mg/gDW),Protein (mg/gDW)
0,132.50774,802.476780,0.000000,65.634675
1,92.15220,846.611177,5.350773,55.885850


### Summary

- amino acids and organic acids content from root exudates available
- generic carbohydrate, lipid and protein content data generated by Schnable lab

In [28]:
biomass.at["GLC_c","root"]=find_average(df_root_content_1_std["Carbohydrate (mg/gDW)"])
biomass.at["L_PHOSPHATIDATE_p","root"]=find_average(df_root_content_1_std["Lipid (mg/gDW)"])
biomass.at["Protein_b","root"]=find_average(df_root_content_1_std["Protein (mg/gDW)"])

### Seed


Gathering sugar, starch and protein content from Table 6 from an [OGTR report](http://www.ogtr.gov.au/internet/ogtr/publishing.nsf/Content/maize-3/$FILE/biologymaize08.pdf)

In [29]:
df_seed_content_1 = pd.DataFrame(data={"metabolite":["protein","starch","sugar"],
                                       "pericarp (g/100gDW)":[3.7,7.3,0.34],
                                       "endosperm (g/100gDW)":[8.0,87.6,0.62],
                                       "embryo (g/100gDW)":[18.4,8.3,10.8]})
df_seed_content_1.set_index("metabolite",drop=True,inplace=True)
df_seed_content_1["pericarp (mg/gDW)"]=df_seed_content_1["pericarp (g/100gDW)"]*10
df_seed_content_1["endosperm (mg/gDW)"]=df_seed_content_1["endosperm (g/100gDW)"]*10
df_seed_content_1["embryo (mg/gDW)"]=df_seed_content_1["embryo (g/100gDW)"]*10
df_seed_content_1

,embryo (g/100gDW),endosperm (g/100gDW),pericarp (g/100gDW),pericarp (mg/gDW),endosperm (mg/gDW),embryo (mg/gDW)
metabolite,,,,,,
protein,18.4,8.00,3.70,37.0,80.0,184.0
starch,8.3,87.60,7.30,73.0,876.0,83.0
sugar,10.8,0.62,0.34,3.4,6.2,108.0


Gathering glucose, fructose, sucrose, starch, protein, amino acids, organic acids  and fatty acids content on boiled sweet corn from Table 7 from an [OGTR report](http://www.ogtr.gov.au/internet/ogtr/publishing.nsf/Content/maize-3/$FILE/biologymaize08.pdf)

In [30]:
df_seed_content_2 = pd.DataFrame(data={"metabolites":["glucose","fructose","sucrose","starch",
                                                      "protein","moisture","C16:0","C18:0",
                                                      "C20:0","C18:1","C20:1","C18:2","C18:3",
                                                      "Ala","Arg","Asp","Cys","Glt","Gly","His",
                                                      "Ile","Leu","Lys","Met","Phe","Pro","Ser",
                                                      "Thr","Try","Tyr","Val","Cit","Mal"],
                                       "g/100gFW":[0.4,0.3,4.6,7.5,4.2,74.0,0.25,0.04,0.01,0.64,
                                                   0.01,1.27,0.02,0.335,0.159,0.257,0.095*2,0.713,
                                                   0.145,0.124,0.155,0.467,0.135,0.105,0.192,0.325,
                                                   0.198,0.199,0.032,0.155,0.224,1,0.2]})
df_seed_content_2.set_index("metabolites",inplace=True,drop=True)

FW = 100
DW = 100 - df_seed_content_2["g/100gFW"]["moisture"]
FW_DW_ratio = float(FW)/DW

df_seed_content_2["mg/gFW"]=df_seed_content_2["g/100gFW"]*10
df_seed_content_2["mg/gDW"]=df_seed_content_2["mg/gFW"]*FW_DW_ratio
df_seed_content_2["mg/gDW"]["moisture"]=0
df_seed_content_2

,g/100gFW,mg/gFW,mg/gDW
metabolites,,,
glucose,0.400,4.00,15.384615
fructose,0.300,3.00,11.538462
sucrose,4.600,46.00,176.923077
starch,7.500,75.00,288.461538
protein,4.200,42.00,161.538462
moisture,74.000,740.00,0.000000
C16:0,0.250,2.50,9.615385
C18:0,0.040,0.40,1.538462
C20:0,0.010,0.10,0.384615


Gathering biomass composition from Seaver et al 2005 endosperm model

In [31]:
#import sbml file (had to remove 6 metabolite and 1 reaction duplicates)
Sam_endo_model = io.sbml.read_sbml_model("Data/Seaver2015_Evidenced-Maize-Endosperm-Model_corrected.sbml")
rxn = Sam_endo_model.reactions.biomass0

In [32]:
df_seed_content_3 = pd.DataFrame(data={"mets":["glucose","fructose","xylose","galactose","malate","citrate",
                                               "Arg","His","Lys","Asp","Glu","Ser","Thr","Asn","Gln","Cys",
                                               "Gly","Pro","Ala","Val","Ile","Leu","Met","Phe","Tyr","Trp"],
                                       "stoichiometry from Seaver et al 2015 leaf model":
                                       [abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd19001_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd30321_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd01487_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00724_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00130_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00137_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00051_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00119_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00039_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00041_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00023_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00054_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00161_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00132_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00053_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00084_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00033_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00129_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00035_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00156_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00322_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00107_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00060_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00066_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00069_c0"))),
                                        abs(rxn.metabolites.get(Sam_endo_model.metabolites.get_by_id("cpd00065_c0")))]})
df_seed_content_3.set_index("mets",drop=True,inplace=True)
df_seed_content_3

,stoichiometry from Seaver et al 2015 leaf model
mets,
glucose,5.204900
fructose,0.171000
xylose,0.216900
galactose,0.063400
malate,0.004207
citrate,0.005624
Arg,0.007300
His,0.009500
Lys,0.010000


In paper, Seaver et al 2005 report starch is the major biomass component. Starch however was missing in the biomass reaction in the model. As a result, biomass composition gathered from biomass reaction is not currently not considered.

Data generated by Schnable lab, email recieved from Lee on 30/09/2019:

In [33]:
df_seed_content_4 = pd.DataFrame(data={"Moisture (%FW)":[54.56,55.54],
                                       "Ash (%FW)":[0,0],
                                       "Protein (%FW)":[4.19,3.78],
                                       "Lipid (%FW)":[0.87,1.09],
                                       "Carbohydrate (%FW)":[40.38,39.59]})
print df_seed_content_4

df_seed_content_4_std = pd.DataFrame(data={"Ash (mg/gDW)":1000*df_seed_content_4["Ash (%FW)"]/(100 - df_seed_content_4["Moisture (%FW)"]),
                                           "Protein (mg/gDW)":1000*df_seed_content_4["Protein (%FW)"]/(100 - df_seed_content_4["Moisture (%FW)"]),
                                           "Lipid (mg/gDW)":1000*df_seed_content_4["Lipid (%FW)"]/(100 - df_seed_content_4["Moisture (%FW)"]),
                                           "Carbohydrate (mg/gDW)":1000*df_seed_content_4["Carbohydrate (%FW)"]/(100 - df_seed_content_4["Moisture (%FW)"])})
df_seed_content_4_std

   Ash (%FW)  Carbohydrate (%FW)  Lipid (%FW)  Moisture (%FW)  Protein (%FW)
0          0               40.38         0.87           54.56           4.19
1          0               39.59         1.09           55.54           3.78


,Ash (mg/gDW),Carbohydrate (mg/gDW),Lipid (mg/gDW),Protein (mg/gDW)
0,0.0,888.644366,19.146127,92.209507
1,0.0,890.463338,24.516419,85.020243


Gathering moisture content, protein, starch, fatty acid, amino acid and cellwall data from FAO website (http://www.fao.org/3/T0395E/T0395E03.htm) 

In [34]:
starch_content = 72.5 #72-73%
lipid_content = 10.5 # 3-18%

In [35]:
df_H2O_protein_temp = pd.DataFrame(data={"variety":["Salpor","Crystalline","Floury",
                                                    "Starchy","Sweet","Pop","Black"],
                                         "Moisture (%FW)":[12.2,10.5,9.6,11.2,9.5,10.4,12.3],
                                         "Protein (%FW)":[5.8,10.3,10.7,9.1,12.9,13.7,5.2]})
df_H2O_protein_temp.set_index("variety",drop=True,inplace=True)
df_H2O_protein_temp

,Moisture (%FW),Protein (%FW)
variety,,
Salpor,12.2,5.8
Crystalline,10.5,10.3
Floury,9.6,10.7
Starchy,11.2,9.1
Sweet,9.5,12.9
Pop,10.4,13.7
Black,12.3,5.2


In [36]:


df_lipid_content_temp = pd.DataFrame(data={"variety":["QPM nutricta","Azotea","Xetzoc",
                                                 "Tropical White","Santa Apolonia"],
                                      "C16:0 (%FA)":[15.71,12.89,11.75,15.49,11.45],
                                      "C18:0 (%FA)":[3.12,2.62,3.54,2.40,3.12],
                                      "C18:1 (%FA)":[36.45,35.63,40.07,34.64,38.02],
                                      "C18:2 (%FA)":[43.38,48.85,44.65,47.47,47.44],
                                      "C18:3 (%FA)":[0.42,0,0,0,0]})
df_lipid_content_temp.set_index("variety",drop=True,inplace=True)
df_lipid_content_temp


,C16:0 (%FA),C18:0 (%FA),C18:1 (%FA),C18:2 (%FA),C18:3 (%FA)
variety,,,,,
QPM nutricta,15.71,3.12,36.45,43.38,0.42
Azotea,12.89,2.62,35.63,48.85,0.00
Xetzoc,11.75,3.54,40.07,44.65,0.00
Tropical White,15.49,2.40,34.64,47.47,0.00
Santa Apolonia,11.45,3.12,38.02,47.44,0.00


In [37]:
df_aminoacid_content_temp = pd.DataFrame(data={"variety":["Cuyuta (white)","SSD (white)","TGY (yellow)",
                                                          "142-48 (yellow)","4251 (hybrid)",
                                                          "HO (high-oil white)","H5 (high-protein yellow)",
                                                          "HP (high-protein white)","Teosinte"],
                                               "N content (assumed g/100gDW)":[1.28,1.37,1.57,1.83,1.31,1.99,2.24,2.91,3.81],
                                               "Ala (g/16g N)":[7.5,12.5,10.3,8.6,8.9,6.4,10.8,9.9,8.5],
                                               "Arg (g/16g N)":[3.5,3.6,4.1,2.9,3.9,4.6,3.6,3.9,2.9],
                                               "Asp (g/16g N)":[6.5,5.8,6.1,6.0,6.2,6.0,6.8,6.1,5.3],
                                               "Cystine (g/16g N)":[1.4,1.4,1.4,1.6,1.6,1.5,1.2,1.4,1.1],
                                               "Glt (g/16g N)":[15.4,14.0,13.5,15.0,14.6,13.9,12.4,12.8,19.9],
                                               "Gly (g/16g N)":[3.1,2.8,2.9,2.6,3.3,3.4,2.6,2.8,2.2],
                                               "His (g/16g N)":[2.1,2.0,2.2,2.1,2.8,2.3,2.0,2.2,1.9],
                                               "Ile (g/16g N)":[2.6,2.7,3.4,3.0,3.3,3.5,3.7,4.0,4.7],
                                               "Leu (g/16g N)":[10.5,12.0,12.2,13.4,12.2,7.8,13.6,15.2,16.8],
                                               "Lys (g/16g N)":[2.8,2.1,2.6,2.3,2.9,3.2,2.1,2.0,1.3],
                                               "Met (g/16g N)":[1.3,1.2,1.0,1.0,1.6,1.2,1.7,1.1,1.2],
                                               "Phe (g/16g N)":[4.4,4.6,4.4,5.4,4.5,2.9,5.3,5.7,5.7],
                                               "Pro (g/16g N)":[8.1,7.4,6.6,9.6,10.3,9.3,8.3,6.8,9.6],
                                               "Ser (g/16g N)":[4.5,4.2,4.6,4.5,4.6,4.8,5.0,5.5,5.2],
                                               "Thr (g/16g N)":[3.1,2.9,3.1,3.0,3.3,3.2,3.1,3.3,3.0],
                                               "Trp (g/16g N)":[0.63,0.47,0.51,0.44,0.49,0.56,0.43,0.44,0.38],
                                               "Tyr (g/16g N)":[2.9,3.0,3.0,3.3,3.4,3.5,3.6,4.1,4.4],
                                               "Val (g/16g N)":[4.1,4.1,4.3,4.0,4.6,2.1,4.3,4.6,4.8]})
df_aminoacid_content_temp.set_index("variety",drop=True,inplace=True)

df_aminoacid_content_temp_std = pd.DataFrame(data={"variety":["Cuyuta (white)","SSD (white)","TGY (yellow)",
                                                          "142-48 (yellow)","4251 (hybrid)",
                                                          "HO (high-oil white)","H5 (high-protein yellow)",
                                                          "HP (high-protein white)","Teosinte"],
                                               "Ala (mg/gDW)":(df_aminoacid_content_temp["Ala (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Arg (mg/gDW)":(df_aminoacid_content_temp["Arg (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Asp (mg/gDW)":(df_aminoacid_content_temp["Asp (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Cystine (mg/gDW)":(df_aminoacid_content_temp["Cystine (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Glt (mg/gDW)":(df_aminoacid_content_temp["Glt (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Gly (mg/gDW)":(df_aminoacid_content_temp["Gly (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "His (mg/gDW)":(df_aminoacid_content_temp["His (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Ile (mg/gDW)":(df_aminoacid_content_temp["Ile (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Leu (mg/gDW)":(df_aminoacid_content_temp["Leu (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Lys (mg/gDW)":(df_aminoacid_content_temp["Lys (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Met (mg/gDW)":(df_aminoacid_content_temp["Met (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Phe (mg/gDW)":(df_aminoacid_content_temp["Phe (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Pro (mg/gDW)":(df_aminoacid_content_temp["Pro (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Ser (mg/gDW)":(df_aminoacid_content_temp["Ser (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Thr (mg/gDW)":(df_aminoacid_content_temp["Thr (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Trp (mg/gDW)":(df_aminoacid_content_temp["Trp (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Tyr (mg/gDW)":(df_aminoacid_content_temp["Tyr (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10,
                                               "Val (mg/gDW)":(df_aminoacid_content_temp["Val (g/16g N)"]/16)* \
                                                            df_aminoacid_content_temp["N content (assumed g/100gDW)"]*10})
df_aminoacid_content_temp_std.set_index("variety",drop=True,inplace=True)
df_aminoacid_content_temp_std

,Ala (mg/gDW),Arg (mg/gDW),Asp (mg/gDW),Cystine (mg/gDW),Glt (mg/gDW),Gly (mg/gDW),His (mg/gDW),Ile (mg/gDW),Leu (mg/gDW),Lys (mg/gDW),Met (mg/gDW),Phe (mg/gDW),Pro (mg/gDW),Ser (mg/gDW),Thr (mg/gDW),Trp (mg/gDW),Tyr (mg/gDW),Val (mg/gDW)
variety,,,,,,,,,,,,,,,,,,
Cuyuta (white),6.000000,2.800000,5.200000,1.120000,12.320000,2.480000,1.680000,2.080000,8.40000,2.240000,1.040000,3.520000,6.480000,3.600000,2.480000,0.504000,2.320000,3.280000
SSD (white),10.703125,3.082500,4.966250,1.198750,11.987500,2.397500,1.712500,2.311875,10.27500,1.798125,1.027500,3.938750,6.336250,3.596250,2.483125,0.402438,2.568750,3.510625
TGY (yellow),10.106875,4.023125,5.985625,1.373750,13.246875,2.845625,2.158750,3.336250,11.97125,2.551250,0.981250,4.317500,6.476250,4.513750,3.041875,0.500438,2.943750,4.219375
142-48 (yellow),9.836250,3.316875,6.862500,1.830000,17.156250,2.973750,2.401875,3.431250,15.32625,2.630625,1.143750,6.176250,10.980000,5.146875,3.431250,0.503250,3.774375,4.575000
4251 (hybrid),7.286875,3.193125,5.076250,1.310000,11.953750,2.701875,2.292500,2.701875,9.98875,2.374375,1.310000,3.684375,8.433125,3.766250,2.701875,0.401188,2.783750,3.766250
HO (high-oil white),7.960000,5.721250,7.462500,1.865625,17.288125,4.228750,2.860625,4.353125,9.70125,3.980000,1.492500,3.606875,11.566875,5.970000,3.980000,0.696500,4.353125,2.611875
H5 (high-protein yellow),15.120000,5.040000,9.520000,1.680000,17.360000,3.640000,2.800000,5.180000,19.04000,2.940000,2.380000,7.420000,11.620000,7.000000,4.340000,0.602000,5.040000,6.020000
HP (high-protein white),18.005625,7.093125,11.094375,2.546250,23.280000,5.092500,4.001250,7.275000,27.64500,3.637500,2.000625,10.366875,12.367500,10.003125,6.001875,0.800250,7.456875,8.366250
Teosinte,20.240625,6.905625,12.620625,2.619375,47.386875,5.238750,4.524375,11.191875,40.00500,3.095625,2.857500,13.573125,22.860000,12.382500,7.143750,0.904875,10.477500,11.430000


In [38]:
df_cellwall_content_temp = pd.DataFrame(data={"variety":[1,2,3,4,5],
                                         "hemicellulose":[4.98,8.05,6.25,9.23,11.44],
                                         "lignin":[0.14,0.12,0.13,0.12,0.14],
                                         "cellwall":[9.1,10.8,12.0,13.1,14.2]})
df_cellwall_content_temp.set_index("variety",drop=True,inplace=True)
df_cellwall_content_temp["cellulose"]=df_cellwall_content_temp["cellwall"]- \
                                        df_cellwall_content_temp["hemicellulose"]- \
                                        df_cellwall_content_temp["lignin"]
df_cellwall_content_temp

,cellwall,hemicellulose,lignin,cellulose
variety,,,,
1,9.1,4.98,0.14,3.98
2,10.8,8.05,0.12,2.63
3,12.0,6.25,0.13,5.62
4,13.1,9.23,0.12,3.75
5,14.2,11.44,0.14,2.62


### Summary

- sugar, starch and protein content reported in table 6 of OGTR (Australian gov.)
- glucose, fructose, sucrose, starch, protein, amino acids, organic acids and fatty acids content on boiled sweet corn. Starch content is considerably lower than Table 7 (because of boiling). Hence this data was not considered.
- sugars, organic acid, amino acid composition gathered from Seaver et al 2005.In the paper, the authors report starch is the major biomass component. Starch however was missing in the biomass reaction in the model. As a result, biomass composition gathered from teh reaction is not currently not considered
- moisture content, protein, starch, fatty acid, amino acid and cellwall data was gathered from UN's FAO website. 

Assumptions

- processing FAO data, in the case of starch, and cell wall, tables were not clear of whether the units were in %FW or %DW, so %FW was considered as other data reported was in %FW. In the case of amino acids, data from a study was reported. The table legend also does not mention is units they are expressed in. Given how the data looks very similar to results from Mosse, Huet and Baudet 1988, the units were assumed to be the same.

In [39]:
FW_DW_ratio = 100/(100-find_average(df_H2O_protein_temp["Moisture (%FW)"]))
biomass.at["Starch_b","seed"]=starch_content*FW_DW_ratio*10
biomass.at["Protein_b","seed"]=find_average(df_H2O_protein_temp["Protein (%FW)"])*FW_DW_ratio*10
biomass.at["PALMITATE_p","seed"]=find_average(df_lipid_content_temp["C16:0 (%FA)"])*find_average(df_seed_content_4_std["Lipid (mg/gDW)"])/100
biomass.at["STEARIC_ACID_p","seed"]=find_average(df_lipid_content_temp["C18:0 (%FA)"])*find_average(df_seed_content_4_std["Lipid (mg/gDW)"])/100
biomass.at["OLEATE_CPD_p","seed"]=find_average(df_lipid_content_temp["C18:1 (%FA)"])*find_average(df_seed_content_4_std["Lipid (mg/gDW)"])/100
biomass.at["Octadecadienoate_p","seed"]=find_average(df_lipid_content_temp["C18:2 (%FA)"])*find_average(df_seed_content_4_std["Lipid (mg/gDW)"])/100
biomass.at["LINOLENIC_ACID_p","seed"]=find_average(df_lipid_content_temp["C18:3 (%FA)"])*find_average(df_seed_content_4_std["Lipid (mg/gDW)"])/100
biomass.at["ARG_c","seed"]=find_average(df_aminoacid_content_temp_std["Arg (mg/gDW)"])
biomass.at["HIS_c","seed"]=find_average(df_aminoacid_content_temp_std["His (mg/gDW)"])
biomass.at["LYS_c","seed"]=find_average(df_aminoacid_content_temp_std["Lys (mg/gDW)"])
biomass.at["sASP_b","seed"]=find_average(df_aminoacid_content_temp_std["Asp (mg/gDW)"])
biomass.at["sGLU_b","seed"]=find_average(df_aminoacid_content_temp_std["Glt (mg/gDW)"])
biomass.at["sSER_b","seed"]=find_average(df_aminoacid_content_temp_std["Ser (mg/gDW)"])
biomass.at["THR_c","seed"]=find_average(df_aminoacid_content_temp_std["Thr (mg/gDW)"])
biomass.at["CYS_c","seed"]=find_average(df_aminoacid_content_temp_std["Cystine (mg/gDW)"])*2
biomass.at["GLY_c","seed"]=find_average(df_aminoacid_content_temp_std["Gly (mg/gDW)"])
biomass.at["PRO_c","seed"]=find_average(df_aminoacid_content_temp_std["Pro (mg/gDW)"])
biomass.at["sALA_b","seed"]=find_average(df_aminoacid_content_temp_std["Ala (mg/gDW)"])
biomass.at["VAL_c","seed"]=find_average(df_aminoacid_content_temp_std["Val (mg/gDW)"])
biomass.at["ILE_c","seed"]=find_average(df_aminoacid_content_temp_std["Ile (mg/gDW)"])
biomass.at["LEU_c","seed"]=find_average(df_aminoacid_content_temp_std["Leu (mg/gDW)"])
biomass.at["MET_c","seed"]=find_average(df_aminoacid_content_temp_std["Met (mg/gDW)"])
biomass.at["PHE_c","seed"]=find_average(df_aminoacid_content_temp_std["Phe (mg/gDW)"])
biomass.at["TYR_c","seed"]=find_average(df_aminoacid_content_temp_std["Tyr (mg/gDW)"])
biomass.at["TRP_c","seed"]=find_average(df_aminoacid_content_temp_std["Trp (mg/gDW)"])
biomass.at["Cellulose_b","seed"]=find_average(df_cellwall_content_temp["cellulose"])*FW_DW_ratio*10

In [40]:
biomass.to_csv("Data/biomass_maize.csv")

In [41]:
biomass

,leaf,root,seed,stem
,,,,
sSUCROSE_b,0.000000,0.000000,0.000000,74.681529
GLC_c,315.062578,824.543979,0.000000,91.563147
FRU_c,46.321473,0.000000,0.000000,0.000000
Starch_b,0.000000,0.000000,812.910460,4.740260
Cellulose_b,255.214725,0.000000,41.710716,108.411215
Xylan_b,0.000000,0.000000,0.000000,37.181070
L_PHOSPHATIDATE_p,0.110183,2.675386,0.000000,0.000000
PHOSPHATIDYL_CHOLINE_r,1.762925,0.000000,0.000000,0.000000
L_1_PHOSPHATIDYL_ETHANOLAMINE_r,0.110183,0.000000,0.000000,0.000000


### References

- Jeannette, E., Reyss, A., Grégory, N., Gantet, P. and Prioul, J. ‐L. (2000) Carbohydrate metabolism in a heat‐girdled maize source leaf. Plant. Cell Environ., 23, 61–69. https://doi.org/10.1046/j.1365-3040.2000.00519.x.
- Gu, Y., He, L., Zhao, C., Wang, F., Yan, B., Gao, Y., Li, Z., Yang, K. and Xu, J. (2017) Biochemical and Transcriptional Regulation of Membrane Lipid Metabolism in Maize Leaves under Low Temperature. Front. Plant Sci., 8, 1–13. Available at: https://doi.org/10.3389/fpls.2017.02053.
- Seaver, S.M.D., Bradbury, L.M.T., Frelin, O., Zarecki, R., Ruppin, E., Hanson, A.D. and Henry, C.S. (2015) Improved evidence-based genome-scale metabolic models for maize leaf, embryo, and endosperm. Front. Plant Sci., 6, 142. http://www.doi.org/10.3389/fpls.2015.00142.
- Abiven, S., Heim, A. and Schmidt, M.W.I. (2011) Lignin content and chemical characteristics in maize and wheat vary between plant organs and growth stages: Consequences for assessing lignin dynamics in soil. Plant Soil, 343, 369–378. https://www.doi.org/10.1007/s11104-011-0725-y.
- Hatfield, R.D. and Marita, J.M. (2017) Maize Development: Cell Wall Changes in Leaves and Sheaths. Am. J. Plant Sci., 08, 1248–1263. https://www.doi.org/10.4236/ajps.2017.86083.
- Sekhon, R.S., Breitzman, M.W., Silva, R.R., Santoro, N., Rooney, W.L., Leon, N. de and Kaeppler, S.M. (2016) Stover Composition in Maize and Sorghum Reveals Remarkable Genetic Variation and Plasticity for Carbohydrate Accumulation. Front. Plant Sci., 7, 1–12. https://www.doi.org/10.3389/fpls.2016.00822.
- Carvalhais, L.C., Dennis, P.G., Fedoseyenko, D., Hajirezaei, M.-R., Borriss, R. and Wirén, N. von (2011) Root exudation of sugars, amino acids, and organic acids by maize as affected by nitrogen, phosphorus, potassium, and iron deficiency. J. Plant Nutr. Soil Sci., 174, 3–11. https://www.doi.org/10.1002/jpln.201000085.
- Ogawa, A., Ando, F., Toyofuku, K. and Kawashima, C. (2009) Sucrose Metabolism for the Development of Seminal Root in Maize Seedlings. Plant Prod. Sci., 12, 9–16. https://www.doi.org/10.1626/pps.12.9.